# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 5 2022 2:27PM,247984,10,8614410,Eywa Pharma Inc.,Released
1,Oct 5 2022 2:19PM,247983,19,ADV80006265,"AdvaGen Pharma, Ltd",Released
2,Oct 5 2022 2:19PM,247983,19,ADV80006266,"AdvaGen Pharma, Ltd",Released
3,Oct 5 2022 2:19PM,247983,19,ADV80006267,"AdvaGen Pharma, Ltd",Released
4,Oct 5 2022 2:19PM,247983,19,ADV80006268,"AdvaGen Pharma, Ltd",Released
5,Oct 5 2022 2:19PM,247983,19,ADV80006269,"AdvaGen Pharma, Ltd",Released
6,Oct 5 2022 2:19PM,247983,19,ADV80006270,"AdvaGen Pharma, Ltd",Released
7,Oct 5 2022 2:19PM,247983,19,ADV80006271,"AdvaGen Pharma, Ltd",Released
8,Oct 5 2022 2:19PM,247983,19,ADV80006272,"AdvaGen Pharma, Ltd",Released
9,Oct 5 2022 2:19PM,247983,19,ADV80006273,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,247980,Released,1
41,247981,Released,3
42,247982,Released,2
43,247983,Released,14
44,247984,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247980,NaN,NaN,1.0
247981,NaN,NaN,3.0
247982,NaN,NaN,2.0
247983,NaN,NaN,14.0
247984,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247841,28.0,30.0,3.0
247886,0.0,6.0,0.0
247888,0.0,0.0,6.0
247889,0.0,0.0,5.0
247892,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247841,28,30,3
247886,0,6,0
247888,0,0,6
247889,0,0,5
247892,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247841,28,30,3
1,247886,0,6,0
2,247888,0,0,6
3,247889,0,0,5
4,247892,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247841,28,30,3
1,247886,,6,
2,247888,,,6
3,247889,,,5
4,247892,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 5 2022 2:27PM,247984,10,Eywa Pharma Inc.
1,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd"
15,Oct 5 2022 2:17PM,247982,10,"CLINUVEL, Inc."
17,Oct 5 2022 2:14PM,247981,10,Emerginnova
20,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix
21,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC"
22,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC"
23,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix
24,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation
50,Oct 5 2022 2:03PM,247976,16,"SHL Pharma, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 5 2022 2:27PM,247984,10,Eywa Pharma Inc.,,,1
1,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd",,,14
2,Oct 5 2022 2:17PM,247982,10,"CLINUVEL, Inc.",,,2
3,Oct 5 2022 2:14PM,247981,10,Emerginnova,,,3
4,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,,,1
5,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC",,,1
6,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC",,,1
7,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix,,,1
8,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation,,,26
9,Oct 5 2022 2:03PM,247976,16,"SHL Pharma, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:27PM,247984,10,Eywa Pharma Inc.,1,,
1,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd",14,,
2,Oct 5 2022 2:17PM,247982,10,"CLINUVEL, Inc.",2,,
3,Oct 5 2022 2:14PM,247981,10,Emerginnova,3,,
4,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1,,
5,Oct 5 2022 2:10PM,247979,10,"ClearSpec, LLC",1,,
6,Oct 5 2022 2:08PM,247978,10,"ClearSpec, LLC",1,,
7,Oct 5 2022 2:07PM,247977,10,Yusen – 3D Matrix,1,,
8,Oct 5 2022 2:04PM,247975,10,ISDIN Corporation,26,,
9,Oct 5 2022 2:03PM,247976,16,"SHL Pharma, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:27PM,247984,10,Eywa Pharma Inc.,1,,
1,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd",14,,
2,Oct 5 2022 2:17PM,247982,10,"CLINUVEL, Inc.",2,,
3,Oct 5 2022 2:14PM,247981,10,Emerginnova,3,,
4,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:27PM,247984,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd",14.0,NaN,NaN
2,Oct 5 2022 2:17PM,247982,10,"CLINUVEL, Inc.",2.0,NaN,NaN
3,Oct 5 2022 2:14PM,247981,10,Emerginnova,3.0,NaN,NaN
4,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 2:27PM,247984,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd",14.0,0.0,0.0
2,Oct 5 2022 2:17PM,247982,10,"CLINUVEL, Inc.",2.0,0.0,0.0
3,Oct 5 2022 2:14PM,247981,10,Emerginnova,3.0,0.0,0.0
4,Oct 5 2022 2:10PM,247980,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5454952,170.0,19.0,1.0
15,495767,5.0,33.0,38.0
16,991639,12.0,6.0,0.0
19,1487829,37.0,21.0,0.0
22,495937,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5454952,170.0,19.0,1.0
1,15,495767,5.0,33.0,38.0
2,16,991639,12.0,6.0,0.0
3,19,1487829,37.0,21.0,0.0
4,22,495937,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,170.0,19.0,1.0
1,15,5.0,33.0,38.0
2,16,12.0,6.0,0.0
3,19,37.0,21.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,170.0
1,15,Released,5.0
2,16,Released,12.0
3,19,Released,37.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,22
Status,,,,,
Completed,1.0,38.0,0.0,0.0,0.0
Executing,19.0,33.0,6.0,21.0,0.0
Released,170.0,5.0,12.0,37.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,22
0,Completed,1.0,38.0,0.0,0.0,0.0
1,Executing,19.0,33.0,6.0,21.0,0.0
2,Released,170.0,5.0,12.0,37.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,22
0,Completed,1.0,38.0,0.0,0.0,0.0
1,Executing,19.0,33.0,6.0,21.0,0.0
2,Released,170.0,5.0,12.0,37.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()